In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 11.7 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292696 sha256=7d69ffbe4e827013581ce38438244dfcb8f410accc812f29289a7e15a957bc23
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#!pip install 'git+https://git@github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install 'git+https://git@github.com/SKTBrain/KoBERT.git@master#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-install-mhzqw786/kobert-tokenizer_68c13157717148cf812a2342f669596e
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-install-mhzqw786/kobert-tokenizer_68c13157717148cf812a2342f669596e
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=4b93d6efd22bd25ad25fbe0ce68d6a1fc933b105ce89b8d7ddda6505cf0c2c81
  Stored in directory: /tmp/pip-ephem-wheel-cache-riq4uvm5/wheels/57/23/c2/a314b5882b5758f13c3738c61007afc9b988681821ff264683
Successfully built kobert_tokenizer


In [ ]:
!wget -O ./ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
!wget -O ./ratings_test.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt

--2023-09-08 05:42:55--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.148.34, 52.219.202.19, 52.219.146.67, ...
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.148.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘./ratings_train.txt’

./ratings_train.txt 100%[===================>]  13.95M  7.26MB/s    in 1.9s    

2023-09-08 05:42:57 (7.26 MB/s) - ‘./ratings_train.txt’ saved [14628807/14628807]

--2023-09-08 05:42:57--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.148.34, 52.219.202.19, 52.219.146.67, ...
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.148.34|:80... connected.
HTTP

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import time
from tqdm.notebook import tqdm
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
import pandas as pd

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
# Torch GPU 설정
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)

In [ ]:
train_data = pd.read_csv('/content/ratings_train.txt', delimiter = '\t')
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
test_data = pd.read_csv('/content/ratings_test.txt', delimiter = '\t')
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [ ]:
data = pd.concat([train_data, test_data])
data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 6.1+ MB


In [ ]:
# data['class'] = [1 if s >=3 else 0 for s in data['rating']] # 긍정이면 1, 부정이면 0
data.drop_duplicates(subset = ['document'], inplace = True) # document 열에서 중복인 내용이 있다면 중복 제거
data['document'] = data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', '') # 정규 표현식 수행
data['document'] = data['document'].str.replace('^ +', '') # 공백은 empty 값으로 변경
data['document'].replace('', np.nan, inplace = True) # 공백은 Null 값으로 변경
data = data.dropna(how = 'any') # Null 값 제거

<ipython-input-10-71215189b057>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data['document'] = data['document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', '') # 정규 표현식 수행
<ipython-input-10-71215189b057>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data['document'] = data['document'].str.replace('^ +', '') # 공백은 empty 값으로 변경


In [ ]:
data_list = []
for q, label in zip(data['document'], data['label']):
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [ ]:
data_list[:5]

[['아더빙진짜짜증나네요목소리', '0'],
 ['흠포스터보고초딩영화줄오버연기조차가볍지않구나', '1'],
 ['너무재밓었다그래서보는것을추천한다', '0'],
 ['교도소이야기구먼솔직히재미는없다평점조정', '0'],
 ['사이몬페그의익살스런연기가돋보였던영화스파이더맨에서늙어보이기만했던커스틴던스트가너무나도이뻐보였다', '1']]

In [ ]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2,
                                               shuffle = True, random_state = 23)

In [ ]:
print(len(dataset_train), len(dataset_test))

154814 38704


모델 불러오기

In [ ]:
#BERTSentenceTransform 수정
class BERTSentenceTransform:
  def __init__(self, tokenizer, max_seq_length, vocab, pad = True, pair = True):
    self._tokenizer = tokenizer
    self._max_seq_length = max_seq_length
    self._pad = pad
    self._pair = pair
    self._vocab = vocab

  def __call__(self, line):
    # convert to unicode
    text_a = line[0]
    if self._pair:
      assert len(line) == 2
      text_b = line[1]

    tokens_a = self._tokenizer.tokenize(text_a)
    tokens_b = None

    if self._pair:
      tokens_b = self._tokenizer(text_b)

    if tokens_b:
      # Modifies 'tokens_a' and 'tokens_b' in place so that the total
      # length is less than the specified length
      # Account for [CLS], [SEP], [SEP] with '-3'
      self._truncate_seq_pair(tokens_a, tokens_b,
                              self._max_seq_length - 3)
    else:
      # Account for [CLS] and [SEP] with '-2'
      if len(tokens_a) > self._max_seq_length - 2:
        tokens_a = tokens_a[0:(self._max_seq_length - 2)]

    vocab = self._vocab
    tokens = []
    tokens.append(vocab.cls_token)
    tokens.extend(tokens_a)
    tokens.append(vocab.sep_token)
    segment_ids = [0] * len(tokens)

    if tokens_b:
      tokens.extend(tokens_b)
      tokens.append(vocab.sep_token)
      segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

    input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

    # The valid length of sentences. Onlt real tokens are attended to
    valid_length = len(input_ids)

    if self._pad:
      # Zero-pad up to the sequence length
      padding_length = self._max_seq_length - valid_length
      # use padding tokens for the rest
      input_ids.extend([vocab[vocab.padding_token]] * padding_length)
      segment_ids.extend([0] * padding_length)

    return np.array(input_ids, dtype = 'int32'), np.array(valid_length, dtype = 'int32'), \
          np.array(segment_ids, dtype = 'int32')

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
    transform = BERTSentenceTransform(bert_tokenizer, max_seq_length = max_len,
                                       vocab = vocab, pad = pad, pair = pair)

    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))

  def __len__(self):
    return (len(self.labels))

In [ ]:
# 하이퍼 파라미터 설정
max_len = 100
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# kobert 공식 git에 있는 get_kobert_model 선언
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path, return_dict=False)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                             padding_token="[PAD]")
    return bertmodel, vocab_b_obj

In [ ]:
import gluonnlp as nlp
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
  def __init__(self,
               bert,
               hidden_size = 768,
               num_classes = 2, # 감정 클래스 수로 조정 긍정/부정 클래스 2개
               dr_rate = None,
               params = None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size, num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p = dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)

    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(),
                          attention_mask = attention_mask.float().to(token_ids.device),
                          return_dict = False)

    if self.dr_rate:
       out = self.dropout(pooler)
    return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
     'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
     'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step,
                                            num_training_steps = t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def calc_accuracy(X, Y):
  max_vals, max_indices = torch.max(X, 1)
  train_acc = (max_indices == Y).sum().data.cpu().numpy() / max_indices.size()[0]
  return train_acc

모델 학습하기

In [ ]:
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

  0%|          | 0/2419 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.8198196291923523 train acc 0.359375
epoch 1 batch id 201 loss 0.48529231548309326 train acc 0.613106343283582
epoch 1 batch id 401 loss 0.2980172038078308 train acc 0.7071773690773068
epoch 1 batch id 601 loss 0.36912593245506287 train acc 0.7434744176372712
epoch 1 batch id 801 loss 0.48168787360191345 train acc 0.7662297128589264
epoch 1 batch id 1001 loss 0.2901333272457123 train acc 0.7823426573426573
epoch 1 batch id 1201 loss 0.23103450238704681 train acc 0.7937135720233139
epoch 1 batch id 1401 loss 0.40418553352355957 train acc 0.801960653104925
epoch 1 batch id 1601 loss 0.3363395035266876 train acc 0.8092598376014991
epoch 1 batch id 1801 loss 0.2670007050037384 train acc 0.8147643670183231
epoch 1 batch id 2001 loss 0.22171419858932495 train acc 0.8200899550224887
epoch 1 batch id 2201 loss 0.3072347640991211 train acc 0.8241921285779191
epoch 1 batch id 2401 loss 0.38979971408843994 train acc 0.8278907226155768
epoch 1 train acc 0.8282708547253597


  0%|          | 0/605 [00:00<?, ?it/s]

epoch 1 test acc 0.8694300964187328
